# Financial fraud detection using machine learning

In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import zipfile
import io

pd.options.display.max_columns = None

In [4]:
# Zip file path
zip_file_path = '../datasets/archive.zip'

# Название CSV файла внутри архива
csv_file_name = 'PS_20174392719_1491204439457_log.csv'

# Загрузка CSV-файла из ZIP-архива в DataFrame
with zipfile.ZipFile(zip_file_path) as z:
    with z.open(csv_file_name) as f:
        df = pd.read_csv(f)

df.head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
